In [117]:
import pandas as pd
from gensim import utils
from gensim.models import FastText
from razdel import tokenize
from huggingface_hub import hf_hub_download
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

In [118]:
data = pd.read_csv("/work/hack/datasets/dataset_1.csv")

for i in range(1, 5):
    new_data = pd.read_csv(f"/work/hack/datasets/dataset_{i}.csv")
    data = pd.concat([data, new_data])

In [119]:
building = pd.read_csv("/work/hack/additional_data/building_20230808.csv")

<ipython-input-119-cf27923f7be3>:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  building = pd.read_csv("/work/hack/additional_data/building_20230808.csv")


In [185]:
class MyIter:
    def __iter__(self):
        for index, row in building.iterrows():
            tokens = list(tokenize(row["short_address"]))
            yield [_.text for _ in tokens]    

model = FastText(vector_size=128, window=5, min_count=1, workers=30, sg=1, negative=10)
model.build_vocab(corpus_iterable=MyIter())
total_examples = model.corpus_count
model.train(corpus_iterable=MyIter(), total_examples=len(building), epochs=20)

(18572982, 54069380)

In [186]:
vector_index = []

for index, row in tqdm(building.iterrows()):
    tokens = list(tokenize(row["short_address"]))
    tokens = [_.text for _ in tokens]
    predict = np.array([model.wv[token] for token in tokens])
    predict = np.mean(predict, axis=0)
    predict = predict / np.linalg.norm(predict)
    vector_index.append((row['id'], predict))

166645it [00:42, 3947.23it/s]


In [187]:
def get_id(text):
    vector = model.wv[text] 
    tokens = list(tokenize(text))
    tokens = [_.text for _ in tokens]
    vector = np.array([model.wv[token] for token in tokens])
    vector = np.mean(vector, axis=0)
    vector = vector / np.linalg.norm(vector).reshape(1, -1) 
    max_sym = -1
    best_id = 0
    step = 10000

    for ind in range(0, len(vector_index), step):
        vectors = np.array([i[1] for i in vector_index[ind: ind + step]]).T
        syms = vector @ vectors
        best_local_ind = np.argmax(syms)
        sym = syms[0][best_local_ind]
        if sym > max_sym:
            max_sym = sym
            best_id = vector_index[ind + best_local_ind][0]
    return best_id, max_sym

In [188]:
cnt = 0
ind = 0
for index, row in tqdm(data.iterrows()):
    ind += 1
    best_id, max_sym = get_id(row["address"])
    #print(best_id, row['target_building_id'])
    if best_id == row['target_building_id']:
        cnt += 1
    if ind == 1000:
        break
print(cnt / 1000)

999it [01:51,  8.97it/s]

0.396


In [184]:
print(cnt / 1000)

0.236


In [191]:
model.save("fasttext.model")

In [190]:
!pwd

/work/hack
